In [41]:
import csv
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sys
plt.style.use('ggplot')
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction import DictVectorizer
plt.rc("font", size=14)
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import seaborn as sns
from sklearn import metrics
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
import ipaddress
from sklearn.preprocessing import OneHotEncoder
from sklearn.svm import LinearSVC
from sklearn.datasets import make_classification
sns.set(style="white")
sns.set(style="whitegrid", color_codes=True)

In [2]:
df_feature = pd.read_csv('bids.csv')

df_label = pd.read_csv('train-2.csv')
df_label.head()

# print(df_feature.columns)
# df_label.address.value_counts()
# df_feature.head()

In [5]:
#join two csv file
df_mix = df_feature.join(df_label.set_index('bidder_id'),on='bidder_id',how = 'right')

#df_mix.groupby('url').sum().sort_values(by='outcome',ascending=False)
#df_mix.groupby('url').count().sort_values(by='outcome',ascending=False)
#df_mix['merchandise'].unique()
#df_mix['country'].unique()
#len(df_mix['device'].unique())
#a=np.asarray(df_mix.groupby('country')['outcome'].sum(),dtype=np.float)
#b=np.asarray(df_mix.groupby('country')['outcome'].count(),dtype=np.float)
#a/b

# df_mix['outcome'].value_counts()
# sns.countplot(x='outcome',data = df_mix,palette="hls")
# plt.show()

# %matplotlib inline
# pd.crosstab(df_mix.country,df_mix.outcome).plot(kind='bar')
# plt.title('Purchase Frequency for Job Title')
# plt.xlabel('merchandise')
# plt.ylabel('Frequency of Purchase')

In [9]:
##data-clean
df_mix =df_mix.dropna()
df_mix_dr = df_mix.drop(['bid_id','bidder_id','auction','payment_account','address'],axis=1)

1      phone1
25     phone4
39    phone35
42    phone38
68    phone57
Name: device, dtype: object

In [10]:
#dummy
cat_vars=[ 'merchandise',
       'country']
for var in cat_vars:
    cat_list='var'+'_'+var
    cat_list = pd.get_dummies(df_mix_dr[var], prefix=var)
    data1=df_mix_dr.join(cat_list)
    df_mix_dr=data1

In [11]:
#convert ip to int
ip = np.asarray(df_mix_dr['ip'])
ip_num=[]
for i in ip:
    ip_num.append(int(ipaddress.ip_address(i)))
df_mix_dr.ip = ip_num

In [12]:
#label encoding
le = preprocessing.LabelEncoder()
ohe = OneHotEncoder( sparse=False ) 
device = df_mix_dr.device
url = df_mix_dr.url
device_code = le.fit_transform(device)
url_code = le.fit_transform(url)
df_mix_dr.device = device_code
df_mix_dr.url = url_code

,merchandise,device,time,country,ip,url,outcome,merchandise_auto parts,merchandise_books and music,merchandise_clothing,...,country_uz,country_ve,country_vi,country_vn,country_ws,country_ye,country_za,country_zm,country_zw,country_zz
1,furniture,1,9.759243e+15,in,852065620,361821,0.0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
25,furniture,3089,9.759243e+15,bd,2470497822,576774,0.0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
39,furniture,2627,9.759243e+15,in,1259073300,336602,0.0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
42,furniture,2907,9.759243e+15,in,3620425458,518197,0.0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
68,furniture,4456,9.759244e+15,in,2261183986,25950,0.0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [13]:
data_vars=df_mix_dr.columns.values.tolist()

In [61]:
to_keep=[i for i in data_vars if i not in cat_vars]

In [63]:
df_fin = df_mix_dr[to_keep]

In [16]:
X = df_fin.loc[:, df_fin.columns != 'outcome']
y = df_fin.loc[:, df_fin.columns == 'outcome']
from imblearn.over_sampling import SMOTE
os = SMOTE(random_state=0)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)
columns = X_train.columns

In [17]:
os_data_X,os_data_y=os.fit_sample(X_train, y_train.values.ravel())
os_data_X = pd.DataFrame(data=os_data_X,columns=columns )
os_data_y= pd.DataFrame(data=os_data_y,columns=['outcome'])
# we can Check the numbers of our data
print("length of oversampled data is ",len(os_data_X))
print("Number of human in oversampled data",len(os_data_y[os_data_y['outcome']==0]))
print("Number of bot",len(os_data_y[os_data_y['outcome']==1]))
print("Proportion of human data in oversampled data is ",len(os_data_y[os_data_y['outcome']==0])/len(os_data_X))
print("Proportion of bot data in oversampled data is ",len(os_data_y[os_data_y['outcome']==1])/len(os_data_X))

length of oversampled data is  3719842
Number of human in oversampled data 1859921
Number of bot 1859921
Proportion of human data in oversampled data is  0.5
Proportion of bot data in oversampled data is  0.5


In [18]:
df_fin_columns = df_fin.columns.values.tolist()
y = ['outcome']
X = [i for i in df_fin_columns if i not in y]

In [19]:
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression()
rfe = RFE(logreg, 20)
rfe = rfe.fit(os_data_X, os_data_y.values.ravel())
print(rfe.support_)
print(rfe.ranking_)

/Users/nightmare/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/nightmare/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/nightmare/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/nightmare/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/nightmare/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will 

/Users/nightmare/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/nightmare/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/nightmare/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/nightmare/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/nightmare/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will 

/Users/nightmare/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/nightmare/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/nightmare/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/nightmare/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/nightmare/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will 

/Users/nightmare/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/nightmare/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/nightmare/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/nightmare/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/nightmare/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will 

/Users/nightmare/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/nightmare/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/nightmare/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/nightmare/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/nightmare/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will 

/Users/nightmare/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/nightmare/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/nightmare/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


[ True  True  True  True False False False  True  True  True  True  True
  True  True False False False False False False False False False False
  True False False False False False False False False False False False
 False False False False False False False False False  True False False
 False False False False False False False False False False False  True
 False False False False False False False False False False False False
 False False False False False False False  True False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False  True  True
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False  True False False False False False False False False False
 False False False False False False False False Fa

In [28]:
#a = [i for i in rfe.support_ if rfe.support_[i] == 'True']
#for i in rfe.support_:
  #  print(rfe.support_[i])
a = []
item_index = np.where(rfe.ranking_==1)
item_index = item_index[0]
for i in item_index:
    a.append(X[i])
a

['device',
 'time',
 'ip',
 'url',
 'merchandise_computers',
 'merchandise_furniture',
 'merchandise_home goods',
 'merchandise_jewelry',
 'merchandise_mobile',
 'merchandise_office equipment',
 'merchandise_sporting goods',
 'country_au',
 'country_ca',
 'country_de',
 'country_gh',
 'country_jp',
 'country_ke',
 'country_ng',
 'country_us',
 'country_za']

In [29]:
cols =['device',
 'time',
 'ip',
 'url',
 'merchandise_computers',
 'merchandise_furniture',
 'merchandise_home goods',
 'merchandise_jewelry',
 'merchandise_mobile',
 'merchandise_office equipment',
 'merchandise_sporting goods',
 'country_au',
 'country_ca',
 'country_de',
 'country_gh',
 'country_jp',
 'country_ke',
 'country_ng',
 'country_us',
 'country_za']

In [30]:
X_1=os_data_X[cols]
y_1=os_data_y['outcome']


In [31]:
import statsmodels.api as sm
logit_model=sm.Logit(y_1,X_1)
result=logit_model.fit()
print(result.summary2())

Optimization terminated successfully.
         Current function value: 0.510663
         Iterations 32
                                          Results: Logit
Model:                          Logit                      Pseudo R-squared:           0.263       
Dependent Variable:             outcome                    AIC:                        3799175.3669
Date:                           2018-11-29 02:49           BIC:                        3799201.6253
No. Observations:               3719842                    Log-Likelihood:             -1.8996e+06 
Df Model:                       1                          LL-Null:                    -2.5784e+06 
Df Residuals:                   3719840                    LLR p-value:                0.0000      
Converged:                      1.0000                     Scale:                      1.0000      
No. Iterations:                 32.0000                                                            
----------------------------------------

In [32]:
X_1_1=X_1.drop(['merchandise_furniture'],axis=1)

In [33]:
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
X_train, X_test, y_train, y_test = train_test_split(X_1_1, y_1, test_size=0.3, random_state=0)
logreg = LogisticRegression()
clf = RandomForestClassifier()
clf.fit(X_train,y_train)
logreg.fit(X_train, y_train)

/Users/nightmare/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/Users/nightmare/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [47]:
y_pred = logreg.predict(X_test)
acc_test = clf.score(X_test,y_test)
acc_test_log = logreg.score(X_test,y_test)

#print('Accuracy of logistic regression classifier on test set: {:.2f}'.format(logreg.score(X_test, y_test)))
print(acc_test)
print(acc_test_log)

0.9472334408348738
0.5005864942340762


In [39]:
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_val_score

#print(classification_report(y_test, y_pred))
cross_val_score(clf,X_test,y_test,scoring='f1',cv=10)

array([0.93888929, 0.93905159, 0.93879596, 0.93679681, 0.93900484,
       0.94008746, 0.93786934, 0.93877106, 0.93870944, 0.93766804])

In [46]:
clf_svc = LinearSVC(random_state=0,tol=1e-5,max_iter=10000)
clf_svc.fit(X_train,y_train)

/Users/nightmare/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:922: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=10000,
     multi_class='ovr', penalty='l2', random_state=0, tol=1e-05, verbose=0)

In [49]:
clf_svc.score(X_test,y_test)

0.49941350576592386

In [45]:
cross_val_score(clf_svc,X_test,y_test,scoring='f1',cv=5)

/Users/nightmare/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:922: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/Users/nightmare/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:922: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/Users/nightmare/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/nightmare/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:922: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/Users/nightmare/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWar

array([0.66718819, 0.        , 0.        , 0.        , 0.        ])

In [56]:
a = [0.93888929, 0.93905159, 0.93879596, 0.93679681, 0.93900484,
       0.94008746, 0.93786934, 0.93877106, 0.93870944, 0.93766804]

In [57]:
np.mean(a)

0.9385643829999999